# To code or not to code  
When faced with a technical challenge or problem solving, conceptual understanding is an important first step, and I almost always start with hand calculations. Once I feel that I have a grasp of the concept, then I turn to a digital approach which often involves using excel or python and running some numbers, and re-running them, and fiddling until I really understand. I always feel like, especially in engineering, that using the right tool for the right job can make a big difference in the results. [Microsoft Excel](https://products.office.com/en-us/excel) or [Libre Calc](https://www.libreoffice.org/discover/calc/) spreadsheet programs are a great way to plug some numbers in and perform simple calculations with nearly no learning curve to get an answer. But as the complexity and quantity of the data and compuation increases, spreadsheets start to not make sense. That's where I feel a code based calculation method is important tool in everyones analysis toolbox. Classicaly, good old pencil and paper was the only way to model and work out problems. But, with the availability of the personal computer, software has allowed us to do more work with less effort. I find it very fascinating to see how people solve problems. This post will explore that idea and hopefully give you an idea to go beyond excel the next time you need to run some numbers.

There are many ways to solve engineering problems. The main tools that I have found critical  are:

 * **hand calculations**  
    * no learning curve
    * low cost
    * slow, tedious and can be error prone (me and algebra :( )
    * forces you to understand 
 * **spreadsheets**  
    * medium learning curve
    * great at simple data storage
    * great for simple calcualtions
    * non-linear / confusing thought process
    * not great at large scale date storage 
    * not repeatable
 * **code**  
    * linear thought process (usually)
    * most powerful for calculations 
    * not great at simple data storage
    * great for large scale data storage and processing 
    * powerful version control ([git](https://git-scm.com/))
    * very repeatable

These are a few methods for solving computational problems. Each one has a time and place, but recognizing when you are trying to put a square peg in a round hole can help keep you sane.


# An Example - python and Excel to understand loans

OK, so we discussed some different approaches to problem solving. Lets look at an example of using an spreadsheet and python code to understand and determine the cost of an loan. Some terms are listed below if you aren't familiar with a compound interest loan.

> apr - annual percentage rate  
> r   - monthly percentge rate, apr/12  
> per - the peroid for which you want to find an interest  
> nper - number of payment  
> pv - present value, the loan amount
> pmt - payment for a loan  
> payments due at the end of the period  

Let's throw some numbers at this. Here I will define some variables that define our loan parameters

In [36]:
############# define input variables #############
n = 24   # term in months
apr = 0.0250   # annual percentage rate
r = apr/12     # monthly percentage rate
baseloan = 30000   # principal amount of loan
downpayment = 5000 
PV = baseloan - downpayment  # present value of the loan

## Hand Calculation  
If we are to hand calculate our loan payments, we can use the following equation (found in Engineering Economy, Blank et al, pg 58)

$$
payment = PV \bigg[  \frac{r(1+r)^n}{(1+r)^n-1} \bigg]
$$


In [37]:
# monthly payments assuming compounding interest
payment = -PV*((r*(1+r)**n)/((1+r)**n-1))
payment

-1069.0098055421547

Easy enough, but what if you want to make a payment table, it would be very tedious by hand. Fortuantley, we have a computer. Since I understand the equation being used, I have no issues with automating this a little more. We can make an spreadsheet table. which is nice visually, didn't take very much time, and allows me to change some parameters.

## Spreadsheet

<iframe width="700" height="807" frameborder="0" scrolling="no" src="https://onedrive.live.com/embed?cid=2DAE54F5236205EE&resid=2DAE54F5236205EE%2125844&authkey=AMSdq_7Zcg_mK_g&em=2&wdAllowInteractivity=False&AllowTyping=True&ActiveCell='loan'!E2&Item='loan'!A1%3AI38&wdHideGridlines=True&wdDownloadButton=True"></iframe>

But, if we want to change, say the term length, well then we have to change the layout of the table and mess with it, so the spreadsheet isn't as robust as we had hoped. How about a code solution? Here are the steps to get some python code going to do the same exact calcuation, and being very robust, and can be used for many different finanical applications.

## Code

In [39]:
############# import python libraries #############
from numpy import *
from pandas import DataFrame, date_range, set_option
set_option('precision',2)

In [40]:
############# calculate output #############
per =              arange(1,n+1)  # months
payment =          pmt(r, n, PV)  # monthly payment
interestpayment =  ipmt(r, per, n, PV)  # monthly interest
principalpayment = ppmt(r, per, n, PV)  # monthly principal
balance =          baseloan - downpayment + per*payment - sum(payment) # monthly balance
dates =            date_range('20160101', periods=n, freq='M')

############# create a table with the output #############
# create a dataframe for simple data management and output
df = DataFrame({'date':dates,
                'month':per,
                'balance':balance,
                'payment':ones(len(per))*payment,
                'interest':interestpayment,
                'principal':principalpayment})
# reorder the columns to match the spreadsheet
df = df[['date','month','balance','payment','interest','principal']]

############# print output #############
print(' monthly payment     = %f' % payment)
print(' total interest cost = %f\n' % sum(interestpayment))   
print(df)

 monthly payment     = -1069.009806
 total interest cost = -656.235333

         date  month   balance  payment  interest  principal
0  2016-01-31      1  25000.00 -1069.01    -52.08   -1016.93
1  2016-02-29      2  23930.99 -1069.01    -49.96   -1019.05
2  2016-03-31      3  22861.98 -1069.01    -47.84   -1021.17
3  2016-04-30      4  21792.97 -1069.01    -45.71   -1023.30
4  2016-05-31      5  20723.96 -1069.01    -43.58   -1025.43
5  2016-06-30      6  19654.95 -1069.01    -41.45   -1027.56
6  2016-07-31      7  18585.94 -1069.01    -39.31   -1029.70
7  2016-08-31      8  17516.93 -1069.01    -37.16   -1031.85
8  2016-09-30      9  16447.92 -1069.01    -35.01   -1034.00
9  2016-10-31     10  15378.91 -1069.01    -32.86   -1036.15
10 2016-11-30     11  14309.90 -1069.01    -30.70   -1038.31
11 2016-12-31     12  13240.89 -1069.01    -28.53   -1040.48
12 2017-01-31     13  12171.88 -1069.01    -26.37   -1042.64
13 2017-02-28     14  11102.87 -1069.01    -24.19   -1044.82
14 2017-03-31

Assuming the user has a basic understanding of how to write a computer program, code is(should be) very straightforward to read. Spreadsheets, like the one shown above can be nebulus and requires a fair bit of digging to see what calcualtions are being made. 
If you need any help with the functions, use the help function to show the inputs

```python
help(pmt)
help(ipmt)
help(ppmt)
```

Try playing around with the code and see what happens when the terms of the loan are changed. The code is more robust to changes, where the spreadsheet has to be reworked

# Summary

A simple demo compared spreadsheets versus code with an example of a common calcualtion problem, loans! When pursuing future projects, think about what tools would be the best for the given problem, as well as you, and possibly your team. 

Stay curious!